In [3]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [2]:
dataset = load_dataset("chirunder/text_messages")

In [3]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [2]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

NameError: name 'df' is not defined

## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [1]:
# Resuelve el task 1 aqui
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

NameError: name 'GPT2Tokenizer' is not defined

In [ ]:
max_length = 50

In [ ]:
def tokenize_text(text):
    tokens = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return tokens['input_ids'][0].tolist()  # Convertir a lista de enteros

df['tokens'] = df['texto'].apply(tokenize_text)

In [ ]:
df_tokenized = df[['n', 'tokens']]
print(df_tokenized.head())

In [ ]:
df_tokenized['tokens'] = df_tokenized['tokens'].apply(lambda x: [int(i) for i in x])

print(df_tokenized.head())

## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [ ]:
# Resuelve el task 2 aqui
db = lancedb.connect("./.lancedb")

table_name = 'tabla'
db.create_table(table_name, schema={
    'n': lancedb.INT,
    'tokens': lancedb.ARRAY(lancedb.INT)
})

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [ ]:
# Resuelve el task 3 aqui
(db["tabla"].search()
    .where("LENGTH(name) > 17")
    .select(["name", "vector"])
    .limit(3)
    .to_pandas())

- Query en SQL equivalente:
SELECT name, vector
FROM tabla1
WHERE LENGTH(name) > 17
LIMIT 3;

- Explicacion: 
Selecciona las columnas 'name' y 'vector' de la tabla que queremos, la condicion es filtrar para mas de 17 caracteres y solo queremos 3 filas.

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [ ]:
# Resuelve el task 4 aqui

most_similar_record = None
min_distance = float('inf')

for record in table.query().execute():
    distance = cosine(record['tokens'], mensaje_tokenized)
    if distance < min_distance:
        min_distance = distance
        most_similar_record = record

# Decodificar el mensaje más similar
most_similar_tokens = most_similar_record['tokens']
decoded_message = tokenizer.decode(most_similar_tokens, skip_special_tokens=True)

print("Mensaje original:", message)
print("Mensaje más similar encontrado:", decoded_message)